In [1]:
# imports
import os
import numpy as np
import json
import pandas as pd

In [3]:
# dir walker

dir1 = 'C:\pepe\share\pepe\projects\bachelor\data\repositories\git\finnqiao\cohort_online_retail'
dir2 = 'C:\pepe\share\pepe\projects\bachelor\data\repositories\git\arvindkarir\retail'
dir3 = 'C:\\pepe\\share\\pepe\\projects\\bachelor\\data\\repositories\\git\\crowdAI\\train-schedule-optimisation-challenge-starter-kit\\'
dir4 = 'C:\\pepe\\share\\pepe\\projects\\bachelor\\data\\repositories\\git\\Lemma1\\DPFE\\'

def dir_walk(dir):
    for root, dirs, files in os.walk(dir):
        #path = root.split(os.sep)
        #print((len(path) - 1) * '-', os.path.basename(root))
        for file in files:
            #print(len(path) * '-', file)
            #if '.ipynb' in file:
            #    print(root, file)
            if '.py' in file:
                print(file)
                
dir_walk(dir4)

base.py
pfe.py


In [5]:
# python code to package parser

fp = 'C:\\pepe\\share\\pepe\\projects\\bachelor\\data\\repositories\\git\\crowdAI\\train-schedule-optimisation-challenge-starter-kit\\utils\\route_penalty.py'

def analyze_pythoncode(code):
    meta = {}
    debug = False
    
    #print(code)
    code = code.split('\n')
    
    libs = []
    for line in code:
        # get libraries
        words = line.split(' ')
        if words[0] == 'import' or words[0] == 'from':
            if debug: print('raw: ' + line.strip())
            lib = line.split(' ')[1].split('.')[0].strip()
            if ',' in lib:
                lib = [elem.strip() for elem in lib.split(',')]
                libs.extend(lib)
                if debug: print('parsed: ['+', '.join(lib) + ']')
            else:
                libs.append(lib)
                if debug: print('parsed: '+lib)

    # make values unique
    libs = np.unique(libs).tolist()
    meta['libs'] = libs
    
    return meta

def analyze_py(filepath):
    meta = {}
    
    
    with open(filepath) as fp:
        content = fp.read()
        
    return analyze_pythoncode(content)
    #print(libs)

meta = analyze_py(fp)

print(meta)

{'libs': ['json', 'pandas']}


In [6]:
# Jupyter Notebook to python code parser

fp = 'C:\\pepe\\share\\pepe\\projects\\bachelor\\data\\repositories\\git\\Lemma1\\DPFE\\DPFE-v0.1.ipynb'

def analyze_ipynb(filepath):
    meta = {}
    py = ''
    debug = True
    
    with open(filepath) as fp:
        c = fp.read()
        c = json.loads(c)
        c = c['cells']
       
    for cell in c:
        for source in cell['source']:
            #print(source.strip())
            py += source
        py += '\n\n'
        
    #print(py)
    return analyze_pythoncode(py)

meta = analyze_ipynb(fp)

print(meta)

{'libs': ['base', 'collections', 'copy', 'datetime', 'joblib', 'matplotlib', 'networkx', 'numpy', 'os', 'pandas', 'pfe', 'pickle', 'random', 'scipy', 'seaborn']}


In [7]:
# python package pattern matching with limited set

ml_libs = ['tensorflow','keras','pytorch','sklearn','nltk','mllib','theano','mxnet','lightgbm','scipy','caffe','cntk','auto_ml','OpenNN','h2o']
matches = set(meta['libs']).intersection(ml_libs)
if not bool(matches):
    matches = {}
print(matches)

{'scipy'}


In [8]:
# python package pattern matching with extended set

ml_libs = pd.read_csv('../data/ml_libraries.csv')
ml_packages = ml_libs['Python Package'].tolist()

matches = set(meta['libs']).intersection(ml_packages)
if not bool(matches):
    matches = {}
print(matches)

{}
